# Quickstart guide - using the Coronado API

The low-level web service API is difficult for external parties to use.  The Coronado API is a set of Python, JVM, and Typescript/JavaScript wrappers designed to streamline integration.

## Installation

The Coronado project is Open Source and licensed under Apache 2.0.  The latest stable code is available from the language-specific repositories like **<a href='https://pypi.org/project/coronado' target='_blank'>PyPI</a>**, Maven Central, and <a href='https://www.npmjs.com/package/coronado' target='_blank'>**NPM**</a>.  Use the standard installer for your target dev tool  to install (`pip`, `mvn`, `npm`, etc.):

In [ ]:
# !pip install -U coronado

---
## Authentication

Authentication is transparent to the Coronado user:

1. Obtain a valid clientID/clientSecret pair from triple
1. Obtain the service's URL from triple; the triple environment
   support multi-tenancy plus sandboxes and production environments with dedicated URLs
1. Initialize the Auth object with this information

The `loadConfig()` utility function can be used for loading the configuration file from a system-dependent standard location.  To view the corresponding location for the system where you're trying the API, run:

### Configuration made simple

In [ ]:
from coronado.auth import SECRETS_FILE_PATH
SECRETS_FILE_PATH

The `Auth` instances use specific arguments for configuration; the `config.json` file and associated methods are available for convenience.  Coronado API users are welcome to use any configuration management they wish, in whatever format.  `Auth` constructors use whatever arguments are passed during initialization.  A typical configuration includes:

In [ ]:
from coronado.auth import loadConfig
conf4Display = loadConfig()
conf4Display['clientID'] = conf4Display['clientID'][-8:]
conf4Display['secret'] = conf4Display['secret'][-16:]
conf4Display

### Instantiate the Auth object

`Auth` instances are long-lived, and can be reused across multiple API calls, across one or more Coronado classes or objects, without worrying about expiration time, regardless of what the OAuth2 policy set up (e.g. 3600 seconds).  `Auth` instances are guaranteed to **always** render a valid JWT token because they renew it behind the scenes if they are expired, without further programmer participation.

To instantiate a new `Auth` object pass the pertinent configuration parameters and the appropriate OAuth scope for the operations using the `Auth`.  You may need to instantiate more than one `Auth` object if the subsequent API calls correspond to different scopes.  See https://api.partners.dev.tripleupdev.com/docs#section/Authentication for more information.  The `Scope` object pre-defines all available scopes:

In [ ]:
from coronado.auth import Scopes

currentScope = Scopes.PUBLISHERS
print('Defined scopes:  %s' % [scope for scope in dir(Scopes) if '__' not in scope])

Instantiate the `Auth` object and keep it around for calls:

In [ ]:
from coronado.auth import Auth

config = loadConfig()
auth = Auth(tokenURL = config['tokenURL'],
            clientID = config['clientID'],
            clientSecret = config['secret'],
            scope = currentScope)

And validate that the instance has all the appropriate attributes set:

In [ ]:
print('token: %s.....426a' % auth.token[:32])
print('token type: %s' % auth.tokenType)

The `auth` object will be ready to use for as long as the process that instantiated is alive.

---
## Using triple objects

triple objects are attribute containers with very little built-in logic.  Their purpose is to make it super-easy to use without having to think about the nitty gritty of URL management, endpoints configuration, and so on.  All objects have the same class attributes and method names, and individual attributes may be accessed using dot-notation (Python, JavaScript, R) or JVM accessor convetions (getters/setters).

### Configuring triple objects to use the correct services provider

1. Initialize the corresponding class to use the right configuration
1. Instantiate an object directly and set its attributes, OR
1. Perform service operations using the corresponding class methods

In [ ]:
from copy import deepcopy
from coronado.publisher import Publisher

Publisher.initialize(config['serviceURL'], auth)

headers = deepcopy(Publisher.headers)
headers['Authorization'] = '%s...69f2' % headers['Authorization'][:64]
print('service: %s' % Publisher._serviceURL)
print('raw token: %s' % Publisher._auth.token[:32])
print('headers: %s' % headers)

### Creating a new Publisher resource

Each individual resource is associated with a persistent instance of a Publisher.  Use a publisher spec, as defined by the API, to create a new Publisher resource.  The publisher spec requires:

- Publisher assumed name (e.g. Acme, Inc.)
- An external ID
- The revenue share as a percentage (1.25, not 0.0125)
- A physical address

Addresses are normalized in the triple ecosystem (in fact, they are also backed by an RDF specification).  That means that the publisher spec also requires a well-formed `Address` object.

#### Working with Address objects

Easy - create an `Address` instance, populate its attributes, and generate the corresponding address spec, compatible with the triple API.

In [ ]:
from coronado.address import Address

address = Address()

print('All address attributes and their types:')
address.listAttributes()

Initialize an address object (semantics) and display it in various application-specific formats:

In [ ]:
address.line1 = '3390 Geary Boulevard'
address.line2 = 'Suite 99'
address.locality = 'San Francisco'
address.province = 'CA'
address.postalCode = '94118'
address.countryCode = 'US'

First, display the addressSpec representation expected by various API objects at creation time:

In [ ]:
address.asSnakeCaseDictionary()

Now, display it as a complete address compatible with the Publisher spec:

In [ ]:
address.complete

Last, display its string representation and compare it against other API objects that don't have a string representation:

In [ ]:
print('address:\n%s' % address)
print('\nauth: %s\n' % auth)

#### Build the Publisher spec

The low-level service API `pubSpec` is a snake_case JSON object with arbitrary attributes set according to the documentation.  Building such an object is trivial:

In [ ]:
import uuid

payload = {
    'address': address.asSnakeCaseDictionary(),
    'assumed_name': 'Acme-%s, Inc.' % uuid.uuid4().hex,   # !!
    'external_id': uuid.uuid4().hex[-12:],   # !!
    'revenue_share': 1.1,

}

**IMPORTANT**

The UUID value mangling of the assumed or externalID value **are not required during normal operation!**  They are present in the Coronado examples and in the unit tests to generate a random value with very low clashing probability because triple **does not support deletion** in any services for the current API version.

**YOU DON NOT NEED TO ADD ANY UUID VALUES TO YOUR ACTUAL OBJECT DEFINITIONS.  They are used only for the examples in this document.**

Resource deletions may lead to data consistency problems for the current implementation.  Therefore, all triple API objects are WORM (write once read many).

#### Creating new objects

Very simple operation:

In [ ]:
newPublisher = Publisher.create(payload)

New objects are assigned a unique object ID, which can be used for cross-linking instances of this object with other objects in the triple API.  To verify this object's ID:

In [ ]:
ref = newPublisher.objID
print('New publisher ID: %s' % newPublisher.objID)

The complete list of a Publisher attributes (or for any TripleObject) can be obtained by using the `listAttributes()` instance method.

In [ ]:
newPublisher.listAttributes()

### Fetching individual objects

Use the object ID to grab an individual object:

In [ ]:
myPublisher = Publisher.byID(ref)
print('%s\n%s' % (myPublisher.assumedName, myPublisher.address.completeAddress))

### Listing objects

The `list()` class method fetches a list of all objects defined in the system.  `list()` methods generate only partial objects, never the complete object.  They are instances of `TripleObject` and code may access their attributes with dot-notation.

In [ ]:
import pandas as pd  # We'll use this for pretty printing

publishers = Publisher.list()
print('total publishers:  %d' % len(publishers))
panel = pd.DataFrame([ publisher.__dict__ for publisher in publishers ])
panel.index = panel.objID
del panel['objID']
panel

### Updating an object

The underlying API implementation allows only for some object attributes to be updated.  In the `Publisher` objects case, only the `assumedName` and `address` attributes are modifiable.  The API is straightforward:

In [ ]:
p = Publisher.byID(39)
originalName = p.assumedName

updatePayload = { 'assumed_name': 'Weyland-Yutani Corporation', }
q = Publisher.updateWith(39, updatePayload)
assert q.assumedName != p.assumedName

print('original name: %s' % originalName)
print('new name: %s' % q.assumedName)

updatePayload['assumed_name'] = originalName
q = Publisher.updateWith(39, updatePayload)
assert q.assumedName == p.assumedName
print('original name restored')

---
## CardProgram

In [ ]:
from coronado.cardprog import CardProgram

CardProgram.initialize(config['serviceURL'], auth)

In [ ]:
import pandas as pd
panel = pd.DataFrame([ p.__dict__ for p in CardProgram.list() ])
panel

In [ ]:
p = CardProgram.list()[0]

In [ ]:
p.listAttributes()

---
# CardAccount

In [ ]:
from coronado.account import CardAccount, CardAccountStatus

import pandas as pd

In [ ]:
CardAccount.initialize(config['serviceURL'], auth)

In [ ]:
panel = pd.DataFrame([ a.__dict__ for a in CardAccount.list() ])
panel.index = panel.objID
del panel['objID']
panel

In [ ]:
dir(Publisher)

In [ ]:
print('Environment: %s; total records: %d' % (Publisher._serviceURL, len(Publisher.list())))

---
## Offer display

In [ ]:
from coronado.auth import loadConfig
from coronado.auth import Auth
from coronado.display import OfferSearchResult
from coronado.display import SERVICE_PATH

import pandas as pd

config = loadConfig()
auth = Auth(tokenURL = config['tokenURL'],
            clientID = config['clientID'],
            clientSecret = config['secret'])
OfferSearchResult.initialize(config['serviceURL'], SERVICE_PATH, auth)

In [ ]:
spec = {
    "proximity_target": {
        "latitude": "40.4604548",
        "longitude": "-79.9215594",
        "radius": 35000
    },
    "card_account_identifier": {
        "card_account_id": '2',
    },
    "text_query": "italian food",
    "page_size": 25,
    "page_offset": 0,
    "apply_filter": {
        "type": "CARD_LINKED"
    }
}
offers = OfferSearchResult.forQuery(spec)

In [ ]:
panel = pd.DataFrame([ o.__dict__ for o in offers ])
panel.index = panel.objID
del panel['objID']
panel